In [9]:
from optimisation_ntn.simulation import Simulation, SimulationConfig
from optimisation_ntn.algorithms.assignment.matrix_based import MatrixBasedAssignment
import numpy as np

sim = Simulation(
    config=SimulationConfig(
        seed=1,
        user_count=10,
        save_results=False,
    )
)
assignment_vector = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
matrix_strategy = MatrixBasedAssignment(sim.network)
matrix_strategy.set_assignment_matrix(assignment_vector)
sim.assignment_strategy = matrix_strategy

while sim.current_time < sim.max_time:
    sim.step()

energy_consumed = sim.system_energy_consumed
print(energy_consumed)

1133.3976311574843


In [10]:
sim.reset()
sim.run_with_assignment(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

(1133.3976311574843, 0.5)